# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#print(file_path_list)

/home/barghy/Documents/Analytics/data-engineering/projects/02-data-modelling-with-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to sparkify
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1: session_playlist

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS session_playlist"
query = query + "(sessionId int, itemInSession int,artist text, song text, length float, PRIMARY KEY (sessionID, itemInSession))"

try:
    session.execute(query)

except Exception as e:
    print(e)                  

In [9]:
# INSERT data into tables from .csv file

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_playlist (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        try:
            session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(length)))
        
        except Exception as e:
            print(e)    

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM session_playlist WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows))
    print(df)
except Exception as e:
    print(e)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


# Query 2: user_session

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session"
query = query + "(userId int, sessionId int, artist text, song text, firstName text, lastName text, itemInSession int, PRIMARY KEY((userId, sessionId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)"

# ASC sorting is defaul - but included to be explicit

try:
    session.execute(query)

except Exception as e:
    print(e)             

In [12]:
# INSERT data into tables from .csv file

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        try:
            session.execute(query, (int(userId), int(sessionId), artist, song, firstName, lastName, int(itemInSession)))

        except Exception as e:
            print(e)    

In [13]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT itemInSession, artist, song, firstName, lastName FROM user_session WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows))
    print(df)
except Exception as e:
    print(e)

   iteminsession             artist  \
0              0   Down To The Bone   
1              1       Three Drives   
2              2  Sebastien Tellier   
3              3      Lonnie Gordon   

                                                song firstname lastname  
0                                 Keep On Keepin' On    Sylvie     Cruz  
1                                        Greece 2000    Sylvie     Cruz  
2                                          Kilometer    Sylvie     Cruz  
3  Catch You Baby (Steve Pitron & Max Sanna Radio...    Sylvie     Cruz  


# Query 3: song_listeners

In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_listeners"
query = query + "(song text, firstName text, lastName text, userId int, PRIMARY KEY(song, userId))"

try:
    session.execute(query)

except Exception as e:
    print(e) 

In [15]:
# INSERT data into tables from .csv file

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line

        try:
            session.execute(query, (song, firstName, lastName, int(userId)))
        
        except Exception as e:
            print(e)    

In [16]:
## Add in the SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName FROM song_listeners WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows))
    print(df)
except Exception as e:
    print(e)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

session.execute("""DROP TABLE session_playlist""")
session.execute("""DROP TABLE user_session""")
session.execute("""DROP TABLE song_listeners""")

### Close the session and cluster connection¶

In [18]:
## Drop the table before closing out the sessions
session.shutdown()
cluster.shutdown()